In [ ]:
from analysis.models.conversation import Conversation, Completion
from pathlib import Path

def conversation_paths_from_evaluation(evaluation_path: Path) -> list[Path]:
    return [
        path for path in (evaluation_path / "llm_completions").iterdir()
        if path.is_dir()
    ]

def load_conversation(completion_logs_path: Path) -> Conversation:
    # The completion logs path should point to a directory that contains individual
    # JSON files representing turns in the conversation. We need to stitch those
    # back together in order.
    turns = []
    for turn_path in completion_logs_path.iterdir():
        with open(turn_path, 'r') as f:
            turn = Completion.model_validate_json(f.read())
            turns.append(turn)
    return Conversation(turns=turns)

In [161]:
EVALUATION_PATH = "/Users/calvin/all-hands/OpenHands/evaluation/evaluation_outputs/outputs/princeton-nlp__SWE-bench_Verified-test/CodeActAgent/claude-3-7-sonnet-20250219_maxiter_250_N_v0.28.1-no-hint-run_1"

conversation_paths = conversation_paths_from_evaluation(Path(EVALUATION_PATH))
conversations = {path: load_conversation(path) for path in conversation_paths}

In [162]:

def condensation_heatmap(conversation: Conversation) -> list[int]:
    statuses: list[list[int]] = []

    for i, completion in enumerate(conversation.turns):


        # The completion is a condensation if it only has a single message
        if len(completion.messages) == 1:

            # We want to compare the turns before and after the condensation
            before_condensation = conversation.turns[i-1]
            after_condensation = conversation.turns[i+1]

            # Convert turns to list of messages hashes for comparison
            before_messages = [hash(message) for message in before_condensation.messages]
            after_messages = [hash(message) for message in after_condensation.messages]

            # To get a sense of what gets dropped in the condensation, map each before
            # message to 1 if it exists in the after messages, and 0 otherwise.
            message_status = [1 if message in after_messages else 0 for message in before_messages]

            statuses.append(message_status)

    status_heatmap = [sum(indices) for indices in zip(*statuses)]

    return status_heatmap

heatmaps = {path: condensation_heatmap(conversation) for path, conversation in conversations.items()}

In [ ]:
import altair as alt
import numpy as np
import pandas as pd

def normalize_heatmap_values(heatmap: list[int]) -> list[float]:
    if not heatmap:
        return heatmap
    
    max_value = max(heatmap)
    return [value / max_value for value in heatmap]

def normalize_heatmap_length(heatmap: list[int], target_length: int) -> list[float]:
    if not heatmap:
        return [1.0] * target_length

    arr = np.array(heatmap)
    return np.interp(np.linspace(0, len(heatmap) - 1, num=target_length), np.arange(len(heatmap)), arr).tolist()

for path, heatmap in heatmaps.items():
    heatmaps[path] = normalize_heatmap_length(normalize_heatmap_values(heatmap), 40)

# Conver the entries in the heatmaps dictionary to a dataframe so we can render it with Altair
rows = []
for path, heatmap in heatmaps.items():
    rows.extend([{
        "path": str(path),
        "index": index,
        "value": value
    } for index, value in enumerate(heatmap)])
df = pd.DataFrame(rows)
# Plot the average heatmap value across all conversations
alt.Chart(df).mark_line().encode(
    alt.X("index:Q").title("Position in context"),
    alt.Y("average(value):Q").title("Likelihood of surviving condensation"),
)

alt.Chart(...)

In [167]:
# Plot the average number of messages in each turn ov the conversation (across all conversations)
rows = []
for path, conversation in conversations.items():
    for index, turn in enumerate(conversation.turns):
        rows.append({
            "path": str(path),
            "index": index,
            "value": len(turn.messages)
        })
df = pd.DataFrame(rows)

df = df.groupby("index").agg({"value": "mean"}).reset_index()

alt.Chart(df).mark_line().encode(
    alt.X("index:Q").title("Turns in the conversation"),
    alt.Y("value:Q").title("Average number of messages"),
)

alt.Chart(...)